In [1]:
from scipy.spatial import KDTree
from geopy.distance import geodesic
import pandas as pd

In [2]:
gh_df = (
    pd.read_csv("geohash_traffic_density_pt_15clusters.csv", usecols=['GEOHASH', 'LATITUDE', 'LONGITUDE'])
)

# Reading Ferries data

# Exclude these as they are the same with other ferry ports
exclude = ["Beykoz İskelesi", 
           "Barbaros Hayrettin Paşa İskelesi", 
           "Haydarpaşa İskelesi",
           "Kabataş İskelesi"]

site_dict = (
    pd.read_csv("datasets/08_tr_istanbul_rail_sea_transportation_locations/tr_ist_ferry_trmnls.csv")
    .drop('ferry_terminal_address', axis=1)
    .query("ferry_terminal not in @exclude") 
    .reset_index(drop=True)
    .set_index('ferry_terminal')
    .T.to_dict()
)

In [3]:
# Unique list of coordinates
coordinates = gh_df[['LATITUDE', 'LONGITUDE']].drop_duplicates().values
coordinates

array([[41.080627, 28.811646],
       [40.987244, 29.108276],
       [41.003723, 29.09729 ],
       ...,
       [40.976257, 29.229126],
       [41.009216, 27.98767 ],
       [41.020203, 27.998657]])

In [4]:
def find_nearest_coordinates(
        target_lat: float,
        target_lon: float,
        coordinates: list[list[float, float]]
        ) -> list[tuple[tuple[float, float], float]]:
    """
    Finds the nearest coordinates to a target latitude and longitude.

    Args:
        target_lat (float): The latitude of the target location.
        target_lon (float): The longitude of the target location.
        coordinates (List[Tuple[float, float]]): A list of tuples representing the coordinates to search from.

    Returns:
        List[Tuple[Tuple[float, float], float]]: A list of tuples containing the nearest coordinates and their distances in kilometers.
    """
    target_coords = (target_lat, target_lon)
    
    # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(coordinates)

    # Calculate the search radius in degrees
    radius_deg = 1 / 111.0  # 1 km radius in degrees

    # Query the KDTree for indices of points within the search radius
    indices = tree.query_ball_point([(target_lat, target_lon)], radius_deg)

    # Calculate the distance between target coordinates and nearest coordinates
    nearest_coordinates = [(coordinates[i], geodesic(target_coords, coordinates[i]).km) for i in indices[0]]

    # Sort the nearest coordinates by distance
    nearest_coordinates.sort(key=lambda x: x[1])  # Sort by distance

    return nearest_coordinates

In [5]:
ferry_list = []
new_list = []

for site in site_dict:

    target_lat = site_dict[site]['latitude']
    target_lon = site_dict[site]['longitude']

    nearest_coords = find_nearest_coordinates(target_lat, target_lon, coordinates)

    for i, (coords, distance) in enumerate(nearest_coords):

        gh = gh_df.loc[gh_df['LATITUDE'].eq(coords[0]) & gh_df['LONGITUDE'].eq(coords[1]), 'GEOHASH'].values[0]

        # Creating two dataframe which will later be merged rows wise for visualization purpose.
        ferry_list.append({
            "type" : "ferry",
            "name" : site,
            "latitude" : target_lat,
            "longitude" : target_lon,
            "nearest" : site
        })

        new_list.append({
            "type" : "geohash",
            "name" : gh,
            "latitude" : coords[0],
            "longitude" : coords[1],
            "nearest" : site
        })

In [6]:
df = (
    pd.concat([pd.DataFrame(new_list), pd.DataFrame(ferry_list)], ignore_index=True)
    .drop_duplicates(ignore_index=True)
)

df

,type,name,latitude,longitude,nearest
0,geohash,sxk9tq,41.080627,29.064331,Anadoluhisarı İskelesi
1,geohash,sxk9tm,41.075134,29.064331,Anadoluhisarı İskelesi
2,geohash,sxk9v2,41.091614,29.064331,Anadoluhisarı İskelesi
3,geohash,sxk3j7,40.976257,28.712769,Avcılar İskelesi
4,geohash,sxk3je,40.976257,28.723755,Avcılar İskelesi
...,...,...,...,...,...
105,ferry,Maltepe İskelesi,40.918176,29.128462,Maltepe İskelesi
106,ferry,Sarıyer İskelesi,41.167160,29.057684,Sarıyer İskelesi
107,ferry,Sütlüce İskelesi,41.046667,28.940688,Sütlüce İskelesi
108,ferry,Üsküdar İskelesi,41.028140,29.015287,Üsküdar İskelesi


In [7]:
# Some GEOHASH has up to 4 sea ferries nearby
(
    df.query("type=='geohash'")
    .groupby('name')['nearest'].nunique()
    .nlargest(10)
)

name
sxk96w    4
sxk96y    4
sxk96t    3
sxk96v    3
sxk96x    3
sxk9tm    3
sxk9tq    3
sxk976    2
sxk977    2
sxk97k    2
Name: nearest, dtype: int64

In [8]:
df.to_csv("datasets/13_gh_proximities/gh_nearby_ferries.csv", index=False)